# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import math
from nltk.tokenize import sent_tokenize


from IPython.display import Markdown, display

In [2]:
import sys
sys.path.append('..')

In [3]:
# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

In [4]:
# import gzip
# import math
# import random
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import GradientBoostingRegressor, \
# GradientBoostingClassifier, RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix, \
# classification_report, make_scorer
# import statsmodels.api as sm

In [5]:
# From this project
# from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLP import WordBag, AboutMovie

In [6]:
%reload_ext autoreload
from subjective_filter import SubjectiveFilter

## Configuration

In [7]:
# Subsampling from Amazon reviews
NB_SAMPLES = 360000 #4000  # up to 200k, then change the input file

data_path = '../../../datasets/'


In [ ]:
# file_name = '360000_balanced_train_test_reviews.pkl'
file_name = '_balanced_pos_neg_train_test_reviews.pkl'

pickle_in = open(data_path + str(NB_SAMPLES) + file_name,"rb")
train_test_dic0 = pickle.load(pickle_in)

In [ ]:
SAMPLE_FRACTION = 0.4

test_dic = {'train': {}, 'test':{}}

for i in ['train','test']:
    for j in ['positive','negative']:
         test_dic[i][j] = train_test_dic0[i][j] \
            .iloc[:math.floor(len(train_test_dic0[i][j].index) * SAMPLE_FRACTION), :] \
            .drop(['reviewerName', 'helpful', 'summary', 'unixReviewTime', 'reviewTime'], axis=1)


In [ ]:
# pickle_in = open(data_path
#                 + 'movie_reviews_144000Pos_Neg_Samples.pkl'
#                 , "rb")
# test_dic = pickle.load(pickle_in)

## Remove objective sentences for case B

In [12]:
OBJ_THRESHOLD = 0.8

In [13]:
%reload_ext autoreload
obj_path = '../obj_subj_dev/'
fit_obj_tf = obj_path + 'fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl'
fit_obj_model = obj_path + 'GBC_300_0.5_5_0.88cv.pkl'
subj_filter = SubjectiveFilter(fit_obj_tf, fit_obj_model)

In [14]:
# for tt in ['train','test']:
#     for pn in ['positive','negative']:
#         print(tt,pn,test_dic[tt][np].shape)
total = 0
for tt in test_dic.values():
    for df in tt.values():
        total += df.shape[0]
        display(df.shape)
print(total)

NameError: name 'test_dic' is not defined

In [ ]:
# test_dic['train']['positive'].iloc[40162:40200,:]

In [ ]:
# from IPython.display import Markdown, display

# for tt in test_dic.values():
#     for df in tt.values():
#         display(df.head(3))

In [ ]:
# type(test_dic['train']['positive']['reviewText'])

In [ ]:
# test = test_dic['train']['positive'].drop(['sentence','words'],axis=1).iloc[:2,:]  # 
# test

In [ ]:
# res = subj_filter.transform(
#             test,
#             'reviewText', 
#             threshold=OBJ_THRESHOLD,
#             debug_level=2)

In [ ]:
# tp = subj_filter.transform(
#             test_dic['train']['positive'].drop(['sentence','words'],axis=1,errors='ignore'),
#             'reviewText', 
#             threshold=OBJ_THRESHOLD,
#             debug_level=2)

In [ ]:
# pickle_out = open(data_path + 'tp_subj_movie_reviews_0.8.pkl', "wb")
# pickle.dump(movie_reviews, pickle_out)

In [ ]:
df.iloc[:5,:]

In [ ]:
res['test']['negative']

In [ ]:
CHUNK_SZ = 5000
sent_dfs = {'train':{},'test':{}}

for ttname, tt in test_dic.items():
    for pn, df in tt.items():
        df = df.drop(['sentence','words'],axis=1,errors='ignore')
        df_list = []
        start = 0
        while start < df.shape[0]:
            end = start + CHUNK_SZ
            df_list.append(
                subj_filter.to_one_sent_per_row(
                    df.iloc[start:end,:]) \
                .rename(columns={"sentence": "reviewText"}))
            start = end

        sent_dfs[ttname][pn] = df_list.pop()
        while len(df_list) > 0:
            sent_dfs[ttname][pn] = pd.merge(df_list.pop(), 
                                            sent_dfs[ttname][pn], how='outer')
#         res[ttname][pn] = reduce(lambda x, y: pd.merge(x, y, on = ['reviewerID','asin']), dfList)


In [ ]:
df1 = sent_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

## Save / Load

In [ ]:
pickle_out = open(data_path + 'sent_dfs_144k.pkl', "wb")
pickle.dump(sent_dfs, pickle_out)

In [8]:
pickle_in = open(data_path
                + 'sent_dfs_144k.pkl'
                , "rb")
sent_dfs = pickle.load(pickle_in)

In [9]:
for ttname, tt in sent_dfs.items():
    for pn, df in tt.items():
        display(df.head(3))

,reviewerID,asin,overall,reviewText
0,A32244V7CQUBD6,B00005QFEK,4.0,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love.
1,A32244V7CQUBD6,B00005QFEN,4.0,This episode pretty much has Hafron and Emmanuelle teleporting to different parts of the world and &quot;doing it&quot;.
2,A33KKMGGVLZ29T,B00005QFER,4.0,This is an intimate concert of Robert Mirabal.


,reviewerID,asin,overall,reviewText
0,A3CFT5LHFCB46K,630362572X,1.0,"In the book The Official Godzilla Compendium, King Kong Vs Godzilla is described as ""The Jaws of the Japanese film Industry"" or ""One of the great monster battles of cinema history""."
1,A1O2XS7W1TUI13,630362572X,1.0,Why would someone buy this pan & scan trash?
2,A2UMZUR8VFEOYB,6303625800,1.0,"I wanted to watch with my grandson, but was so disappointed.Story line and content at times was terrible."


,reviewerID,asin,overall,reviewText
0,A2D832OA6Q5ZAS,B00005QFFP,4.0,What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known.
1,A1JF78EP4GPAOO,B00005QG2N,4.0,The concert is fantastic as are the videos and Cradle of Fear trailer.
2,AQ7K219573Z8P,B00005QIVF,4.0,"Lau Ching Wan gives a strong but over-the-top performance as a very rough and mean natured gangster just out of prison, who has been abandoned by his gang."


,reviewerID,asin,overall,reviewText
0,A175U4QHQUXC69,6303625800,1.0,The DVD would not play anything but the selection page for color or black and white version.
1,A46HSTZBUZ5ZH,6303625800,1.0,"Being a definitive fan of Laurel and Hardy, and having a special place in my heart for ""Babes In Toyland"" (aka ""March of the Wooden Soldiers""), I was excited to add another version to my DVD collection.Alas, the colorization from Legend Films is a big disappointment."
2,A3KEABH9BZ1MPB,6303625800,1.0,The adults found it very boring and the children ages 8 and 10 said it was just ok


In [16]:
df1 = sent_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

,reviewerID,asin,overall,reviewText
0,A32244V7CQUBD6,B00005QFEK,4.0,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love.
5001,A32244V7CQUBD6,B00005QFEK,4.0,It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.


## Remove obj sentences

In [ ]:
# Crashes kernel
# DROP_RATIO = 0.15

# subj_dfs = {'train':{},'test':{}}

# for ttname, tt in sent_dfs.items():
#     for pn, df in tt.items():
#            subj_dfs[ttname][pn] = subj_filter.transform(
#                     df,
#                     'reviewText', 
#                     threshold = None,
#                     debug_level=0,
#                     switch=True,
#                     drop_ratio=DROP_RATIO)

In [15]:
%reload_ext autoreload
from subjective_filter import SubjectiveFilter

DROP_RATIO = 0.15
CHUNK_SZ = 50000
subj_dfs = {'train':{},'test':{}}

for ttname, tt in sent_dfs.items():
    for pn, df in tt.items():
        df_list = []
        start = 0
        while start < df.shape[0]:
            end = start + CHUNK_SZ
            df_list.append(
                subj_filter.transform(
                    df.iloc[start:end,:],
                    'reviewText', 
                    threshold = None,
                    debug_level=0,
                    switch=True,
                    drop_ratio=DROP_RATIO))      
            start = end

        subj_dfs[ttname][pn] = df_list.pop()
        while len(df_list) > 0:
            subj_dfs[ttname][pn] = pd.merge(df_list.pop(), 
                                            subj_dfs[ttname][pn], how='outer')

#### => Removed 7500 (15%) objective sentences

#### => Removed 7500 (15%) objective sentences

#### => Removed 7500 (15%) objective sentences

#### => Removed 7500 (15%) objective sentences

#### => Removed 7500 (15%) objective sentences

...

#### => Removed 7500 (15%) objective sentences

#### => Removed 1481 (15%) objective sentences

#### => Removed 7500 (15%) objective sentences

#### => Removed 7500 (15%) objective sentences

#### => Removed 143 (15%) objective sentences

In [17]:
df1 = subj_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

,reviewerID,asin,reviewText,overall
5410,A32244V7CQUBD6,B00005QFEK,It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode. This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love.,4.0


In [ ]:
# TEST connection
# CHUNK_SZ = 5
# subj_dfs = {'train':{},'test':{}}
# df_test = pd.DataFrame({'A': [1,2,3,4,5,6], 'B':[1,2,3,4,5,6]})

# df_list = []
# start = 0
# while start < df_test.shape[0]:
#     end = start + CHUNK_SZ
#     df_list.append(df_test.iloc[start:end,:])
#     start = end

# out = df_list.pop()
# while len(df_list) > 0:
#     out = pd.merge(df_list.pop(), out, how='outer')
# out

In [ ]:
# for ttname, tt in sent_dfs.items():
#     for pn, df in tt.items():
#         subj_filter.transform(
#             df.rename(columns={"reviewText": "sentence"}),
#             'reviewText', 
#             threshold = None,
#             debug_level=0,
#             switch=True,
#             drop_ratio=0.15)
#         .rename(columns={"sentence": "reviewText"})

In [ ]:
total = 0
old_total = 0
for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        print('\n {} {}'.format(tt,pn))
        tot = test_dic[tt][pn].shape[0]
        new = subj_dfs[tt][pn].shape[0]
        dropped = tot - new
        print('Dropped: {0} ({1:.1%})'.format(dropped, dropped/tot))
        total += new
        old_total += tot
print('\nOld total:', old_total)
print('New total:', total)
print('Removed a total of:', old_total - total)

In [20]:
pickle_out = open(data_path
                    + 'reviews_wout_most_subj_' + str(DROP_RATIO) + '.pkl'
                    , "wb")
pickle.dump(subj_dfs, pickle_out)
pickle_out.close()